In [2]:
print('hello')

hello


In [3]:
import asyncio
from typing import Optional

from fastapi import HTTPException
from datetime import datetime
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

from dotenv import load_dotenv
load_dotenv()


d:\Miniconda\envs\smc\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [4]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, streaming= True)

In [7]:
async def invoke_chain(prompt: PromptTemplate, inputs: dict) -> str:
    chain = prompt | llm | StrOutputParser()
    res = await chain.ainvoke(inputs)
    return res

In [5]:
async def check_topic_relevance(user_question: str, topic_name: str) -> bool:
    """
    Kiểm tra xem câu hỏi có liên quan đến chủ đề hay không.
    Trả về True nếu liên quan, False nếu không liên quan.
    """
    if not topic_name:
        return True  
    
    prompt = PromptTemplate(
        template="""
Chủ đề của session: {topic_name}
Câu hỏi của người dùng: {user_question}

Hãy đánh giá xem câu hỏi của người dùng có liên quan đến chủ đề "{topic_name}" hay không.

Trả lời CHỈ bằng một từ: "CÓ" nếu câu hỏi liên quan đến chủ đề, "KHÔNG" nếu không liên quan.
Không giải thích thêm, chỉ trả lời "CÓ" hoặc "KHÔNG".
""",
        input_variables=["topic_name", "user_question"],
    )
    
    result = await invoke_chain(prompt, {
        "topic_name": topic_name,
        "user_question": user_question
    })
    
    result_upper = result.strip().upper()
    return "CÓ" in result_upper or "YES" in result_upper or "TRUE" in result_upper

In [11]:
res = await check_topic_relevance("Đề bài: Tính tổng 2 số", "asdfadsf")
print(f"Relevance: {res}")
type(res)

Relevance: False


bool

In [ ]:
async def func_chatbot_qa_non_stream(question: str, answer: str, user_question: str, topic_name: Optional[str] = None) -> str:
    """Non-streaming version - returns complete result"""
    if que:
        is_relevant = await check_topic_relevance(user_question, question)
        if not is_relevant:
            return f"Xin lỗi, câu hỏi của bạn không liên quan đến chủ đề **{topic_name}** của session này. Vui lòng hỏi các câu hỏi liên quan đến chủ đề này."
    
    prompt = PromptTemplate(
        template="""
Đề bài: {question}
Bài code của sinh viên: {answer}

Câu hỏi của sinh viên: {user_question}
{focus_topic}

Bạn là trợ lý AI hỗ trợ sinh viên học lập trình. Hãy trả lời câu hỏi của sinh viên dựa trên ngữ cảnh đề bài và bài code của họ.

QUY TẮC QUAN TRỌNG:
- TUYỆT ĐỐI KHÔNG đưa ra đáp án hoàn chỉnh hoặc code mẫu giải bài tập
- TUYỆT ĐỐI KHÔNG viết lại toàn bộ code đúng cho sinh viên
- CHỈ hướng dẫn, gợi ý hướng đi, giải thích khái niệm, phân tích logic
- Nếu sinh viên hỏi về lỗi cụ thể trong code của họ, CHỈ chỉ ra lỗi và gợi ý cách suy nghĩ để sửa, KHÔNG sửa code giúp họ
- Nếu sinh viên hỏi về khái niệm (for, while, if, function, biến...), hãy giải thích rõ ràng với ví dụ đơn giản KHÔNG LIÊN QUAN đến bài tập của họ
- Nếu sinh viên hỏi "làm sao để...", hãy hướng dẫn tư duy và các bước cần làm, KHÔNG viết code mẫu
- Khuyến khích sinh viên tự suy nghĩ và thử nghiệm
- Giới hạn ngữ cảnh trong phạm vi đề bài và code của sinh viên
- Không cần chào 
Trả lời bằng tiếng Việt, xưng hô "bạn", giọng điệu thân thiện, động viên. Trả về kết quả dạng Markdown để dễ đọc.
""",
        input_variables=["question", "answer", "user_question", "focus_topic"],
    )
    
    focus_topic_text = f"\nLƯU Ý: Session này tập trung vào chủ đề **{topic_name}**. Hãy đảm bảo câu trả lời liên quan đến chủ đề này." if topic_name else ""
    
    return await invoke_chain(prompt, {
        "question": question,
        "answer": answer,
        "user_question": user_question,
        "focus_topic": focus_topic_text
    })

In [18]:
ss  = await func_chatbot_qa_non_stream("Đề bài: Tính tổng 2 số", "def sum(a, b): return a + b", "sdfsd")

In [19]:
ss

'Có vẻ như bạn đang muốn tính tổng của hai số. Mình thấy bạn đã viết một hàm `sum` khá đơn giản và đúng về mặt logic. Tuy nhiên, nếu bạn muốn hiểu rõ hơn về cách hoạt động của hàm này hoặc cách sử dụng nó, mình có thể gợi ý một số điều bạn có thể làm:\n\n1. **Kiểm tra kiểu dữ liệu**: Bạn có thể kiểm tra xem hai tham số `a` và `b` có phải là số hay không trước khi thực hiện phép cộng. Điều này giúp đảm bảo rằng hàm của bạn hoạt động như mong đợi.\n\n2. **Gọi hàm**: Hãy thử gọi hàm `sum` với các giá trị khác nhau để xem kết quả. Bạn có thể thử với số dương, số âm, hoặc thậm chí là số thập phân.\n\n3. **Xử lý ngoại lệ**: Cân nhắc việc xử lý các trường hợp mà người dùng có thể nhập vào kiểu dữ liệu không hợp lệ (như chuỗi hoặc danh sách). Bạn có thể nghĩ đến việc sử dụng `try` và `except` để quản lý những lỗi này.\n\n4. **Giải thích tên hàm**: Bạn có thể xem xét việc đổi tên hàm `sum` thành một cái tên khác, vì `sum` là tên của một hàm có sẵn trong Python, điều này có thể gây nhầm lẫn.\n\n